In [253]:
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag

In [254]:
df = pd.read_csv('recipes.csv',index_col=0)

In [255]:
df.head().T

,0,1,2,3,4
recipe_name,Apple-Cranberry Crostada,Apple Pie by Grandma Ople,Sarah's Homemade Applesauce,Apple Crisp,Apple Pie Filling
prep_time,NaN,30 mins,10 mins,30 mins,20 mins
cook_time,NaN,1 hrs,15 mins,45 mins,20 mins
total_time,NaN,1 hrs 30 mins,25 mins,1 hrs 15 mins,2 hrs 40 mins
servings,8,8,4,12,40
yield,6 to 8 - servings,1 9-inch pie,NaN,1 9x13-inch pan,5 9-inch pies
ingredients,"3 tablespoons butter, 2 pounds Granny Smith ap...","8 small Granny Smith apples, or as needed, ½ c...","4 apples - peeled, cored and chopped, ¾ cup w...","10 cups all-purpose apples, peeled, cored and ...","18 cups thinly sliced apples, 3 tablespoons le..."
directions,Heat butter in a large skillet over medium-hig...,"Peel and core apples, then thinly slice. Set a...","Combine apples, water, sugar, and cinnamon in ...",Preheat the oven to 350 degrees F (175 degrees...,Toss apples with lemon juice in a large bowl a...
rating,4.4,4.8,4.8,4.7,4.7
url,https://www.allrecipes.com/recipe/76931/apple-...,https://www.allrecipes.com/recipe/12682/apple-...,https://www.allrecipes.com/recipe/51301/sarahs...,https://www.allrecipes.com/recipe/12409/apple-...,https://www.allrecipes.com/recipe/12681/apple-...


In [256]:
mods_l_1 = ['baked', 'blanched', 'blackened', 'braised', 'breaded', 'broiled', 'caramelized', 'charred', 'fermented', 'fried',
         'glazed', 'infused', 'marinated', 'poached', 'roasted', 'sauteed', 'seared', 'smoked', 'whipped']

# -ed adjectives.  Found another good list online.
# Some overlap in these lists, which is why we're using sets!
mods_l_2 = ['diced', 'battered', 'blackened', 'blanched', 'blended', 'boiled', 'boned', 'braised', 'brewed', 'broiled',
           'browned', 'butterflied', 'candied', 'canned', 'caramelized', 'charred', 'chilled', 'chopped', 'clarified', 
           'condensed', 'creamed', 'crystalized', 'curdled', 'cured', 'curried', 'dehydrated', 'deviled', 'diluted',
           'dredged', 'drenched', 'dried', 'drizzled', 'dry roasted', 'dusted', 'escalloped', 'evaporated', 'fermented',
           'filled', 'folded', 'freeze dried', 'fricaseed', 'fried', 'glazed', 'granulated', 'grated', 'griddled', 'grilled',
           'hardboiled', 'homogenized', 'kneaded', 'malted', 'mashed', 'minced', 'mixed', 'medium', 'small', 'large', 
           'packed', 'pan-fried', 'parboiled', 'parched', 'pasteurized', 'peppered', 'pickled', 'powdered', 'preserved',
           'pulverized', 'pureed', 'redolent', 'reduced', 'refrigerated', 'chilled', 'roasted', 'rolled', 'salted',
           'saturated', 'scalded', 'scorched', 'scrambled', 'seared', 'seasoned', 'shredded', 'skimmed', 'sliced',
           'slivered', 'smothered', 'soaked', 'soft-boiled', 'hard-boiled', 'stewed', 'stuffed', 'toasted', 'whipped',
           'wilted', 'wrapped']

In [257]:
units_l_1 = ['l', 'dl', 'milliliter', 'liter', 'deciliter', 'teaspoon', 't.', 'tsp.',
             'milliliters', 'liters', 'deciliters', 'teaspoons', 't.', 'tsp.',
            'tablespoon', 'T.', 'tbsp.', 'ounce', 'fl oz', 'cup', 'c.', 'pint', 'pt.',
            'tablespoons', 'ounces', 'fl ozs', 'cups', 'pints', 'quarts', 'gallons', 'grams', 'kilograms',
            'quart', 'qt.', 'gallon', 'gal', 'mg', 'milligram', 'g', 'gram', 'kg', 'kilogram', 'milligrams',
            'pound', 'lb', 'ounce', 'oz', 'count', 'pints', 'quarts', 'cups', 'tablespoons',
            'pounds', 'lbs', 'ounces', 'units', 'drops', 'tsps.', 'tbsps.', 'Ts.', 'ts.','inch'
            'teaspoons', 'dash', 'pinch', 'drop', 'dram', 'smidgeon', 'dashes', 'pinches', 'drops', 
             'drams', 'smidgeons', ]

In [258]:
df['ingredients'][0]

'3 tablespoons butter, 2 pounds Granny Smith apples (or other firm, crisp apples), peeled, quartered, cored and sliced 1/4-inch thick, 1 pound Macintosh apples (or other soft-textured apples that fall apart when cooked), peeled, quartered, cored, and sliced 1/4-inch thick, ½ cup sugar, ½ cup dried cranberries, 1 sheet frozen puff pastry, thawed but still cold (follow package directions), 1  egg white, lightly beaten, 1 tablespoon sugar, 1 cup Optional: Ice cream or lightly sweetened whipped cream'

In [259]:
# Define a list of stopwords
stopwords = mods_l_1 + mods_l_2 + units_l_1
stopwords += ["½","raw","'","]","[","(",")","ground"]

# Function to filter non-verb words
def filter_non_verbs(text):
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    non_verb_words = []
    non_verb_words += [word for word, pos in tagged_words if pos in ['NNS','NN'] and \
                      word not in stopwords]
    #remove duplicates
    result = []
    [result.append(x) for x in non_verb_words if x not in result]
    return result

# Apply the function to the 'Text' column and create a new 'Filtered_Text' column
df['Filtered_Text'] = df['ingredients'].apply(filter_non_verbs)

# Flatten the lists in the 'Filtered_Text' column if needed
#df['Filtered_Text'] = df['Filtered_Text'].apply(lambda x: ','.join(x))
#df['Filtered_Text'] = df['Filtered_Text'].split(',',expand=True)

In [260]:
#df['Filtered_Text']=df['Filtered_Text'].str.replace("'","")

In [261]:
df.head(1).T

,0
recipe_name,Apple-Cranberry Crostada
prep_time,NaN
cook_time,NaN
total_time,NaN
servings,8
yield,6 to 8 - servings
ingredients,"3 tablespoons butter, 2 pounds Granny Smith ap..."
directions,Heat butter in a large skillet over medium-hig...
rating,4.4
url,https://www.allrecipes.com/recipe/76931/apple-...


In [262]:
df['id']=range(1,len(df)+1)
df=df['Filtered_Text']

In [263]:
df.to_csv('filtered_ingred.csv')